In [4]:
import re

In [7]:
texto ="o jogador "

In [ ]:
#look ahead look behind nao consome a frase

# Backtracking

https://www.regular-expressions.info/catastrophic.html

In [8]:
test_string

NameError: name 'test_string' is not defined

In [ ]:
pattern = r'x+x+y'

In [ ]:
import re
matcher = re.compile(pattern)

In [27]:
%%time
test_string = 'x'*100000
#test_string += 'y'
matcher.fullmatch(test_string)

Wall time: 7.85 s


# Processando textos reais

Vamos trabalhar com um conjunto de textos obtidos da Wikipedia em portugues (https://dumps.wikimedia.org/ptwiki/20210301/). O conjunto original de dados é bem grande (1.8 GB compactado), então eu criei um arquivo reduzido da seguinte forma:

- Li os registros em formato XML (é assim que estão armazenados no *dump*)

- Removi todos os registros que não diziam respeito a um artigo, mas que eram referentes a redirecionamento de pagina e outras meta-informações.

- Removi todos os campos de cada registro exceto titulo e corpo.

- Selecionei apenas $1\%$ dos registros restantes.

- Gravei cada registro como uma linha de texto no formato JSON em um arquivo dump_small.jsonln ("JSON lines")

Vamos ler esse arquivo e ver seu conteudo:

In [135]:
import json

data = []
with open('dump_small.jsonln', 'r') as file:
    for line in file:
        data.append(json.loads(line))
        
print(f'Numero de documentos: {len(data)}')

Numero de documentos: 11225


In [2]:
print(data[0].keys())

dict_keys(['title', 'body'])


In [3]:
print(data[0]['title'])

Alexandre (nome)


In [11]:
print(data[0]['body'])

{{Multitag|mnot|rev|data=outubro de 2019}}
{{Info/Nome
|imagem= Napoli BW 2013-05-16 16-24-01.jpg
|legenda= [[Alexandre, o Grande]], uma das pessoas notórias de nome '''Alexandre'''.
|gênero= Masculino
|significado= Protetor do Homem
|origem= Do [[Língua grega|grego]] Αλέξανδρος
|relativos= Alex; Alexander; Alessandro; Sandro
}}
'''Alexandre''' é um [[prenome]] popular da [[língua portuguesa]]. É [[cognato]] ao nome [[Alexander]], da [[língua inglesa]]. Em [[países lusófonos]], pessoas chamadas Alexandre são normalmente [[Alcunha|apelidadas]] de [[Alex]].

== Origem ==
O nome deriva do [[latim]] ''Alexander'', [[romanização]] do nome [[Língua grega|grego]] Αλέξανδρος (''Aléksandros''). [[Etimologia|Etimologicamente]], o nome seria um composto do [[verbo]] ''ἀλέξειν'' (''aléksein''), "defender", e o [[substantivo]] ''ἀνδρός'' (''andrós''), [[genitivo]] de ''ἀνήρ'' (''anēr''), "homem". Assim, pode ser traduzido como "protetor da Humanidade".{{carece de fontes|data=abril de 2017}}

O term

---

Como vocês podem ver, o texto está cheio de caracteres de estruturação de documento.

---

**Atividade:** Consulte a documentação da Wikipedia para descobrir o que são as marcações `[[`, `]]`, `{{` e ```}}```

**R:** 
https://en.wikipedia.org/wiki/Help:Wikitext


- [[ ]] => link para outra página da Wikipédia (fazer um hiperlink)
- {{ }} => são templates que substitui por rum html desejado

---

Vamos explorar o corpus para minerar itens de interesse.

**Exercicio:** Faça uma função que retorna uma lista com todos os links html de um texto deste corpus.

In [144]:
def links_html(data):
    list_link = []
    href = re.compile(r"(?<=\[\[).*(?=\]\])")  
    matches = href.finditer(data)
    for match in matches:
        list_link.append(match.group())
        
    return list_link
        
# links_html(data[0]["body"])

**Exercicio:** Faça uma função que recebe um texto do corpus e substitui todas as ocorrências de links da Wikipedia (textos entre os tags `[[` e `]]`) por texto simples, e retorna o documento limpo. Por exemplo, quando encontrar algo como `[[Etimologia|Etimologicamente]]` substituir por `Etimologicamente`. Você pode supor que os links não são *aninháveis*, como por exemplo `[[blablabla[[etc]]blebleble]]`.

In [138]:
def clean_doc(data):
    href = re.compile(r"\[\[(?!.+?:)([^\]\[]+)\|([^\]\[]+)\]\]")
    matches = href.finditer(data)
    for match in matches:
        data= data.replace(match.group(), match.group(2))
    return data

# clean_doc(data[0]["body"])

**Exercicio**: Faça uma função que recebe um texto do corpus e remove todas as ocorrências de referências (textos entre os tags `<ref>` e `</ref>`).

In [ ]:
dado = data[0]["body"]

In [143]:
def clean_ref(dado):
    ref = re.compile(r"(?<=.)<ref>.*</ref>(?=)")
    matches = ref.finditer(dado)
    for match in matches:
        dado= dado.replace(match.group(), "")
    return dado

# clean_ref(data[0]["body"])

**Exercicio:** Faça uma função que recebe um texto do corpus e remove todos os templates (textos entre os tags `{{` e `}}`). Este exercício é desafiante: *os templates podem ser aninhados*! E agora, como proceder?

**Exercicio:** Usando os codigos desenvolvidos acima, faça uma função que recebe um texto do corpus e retorna a sua versão limpa.

**Exercicio:** Limpe todos os documentos e explore os resultados para ver o que mais dá para limpar. Nosso objetivo é ter uma coletânea de textos limpos para poder criar um vocabulário da língua portuguesa!

# Criando um tokenizador

Agora que temos um conjunto de documentos limpos, vamos analisar esses documentos para:

- Separar os documentos em palavras

- Separar os documentos em frases

**Exercicio:** Faça uma função que separa os documentos em palavras usando simplesmente os espaços em branco como indicador de separação.

In [146]:

documents = data[0]['body']

def separaEspaco(texto):
    palavras = texto.split(' ')
    return palavras
# separaEspaco(documents)

**Exercicio:** Agora analise os resultados obtidos: essa separação em palavras está perfeita? Certamente não:

- Um problema óbvio é o seguinte: palavras ao final de uma sentença podem carregar consigo a pontuação da frase. 

- Mais ainda, algumas palavras no meio das sentenças podem ter vírgula e ponto-e-vírgula juntos.

- Sequências de emojis podem aparecer, e cada emoji deve ser considerado como uma palavra separada

Faça uma função que recebe um documento e faz uma separação melhor.

In [148]:
def separacaoMelhor(comPontuacao):
    semPontuacao = []
    for palavra in comPontuacao:
        palavrasSemPontuacao = re.sub(r"[^\w\s]|\n", "", palavra)
        semPontuacao.append(palavrasSemPontuacao)
    return semPontuacao

# separacaoMelhor(separaEspaco(documents))
